In [12]:
import numpy as np
import pandas as pd


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

Загрузка файла

In [28]:
bn = pd.read_csv('homework.csv')
bn = bn.drop(['ID'], axis=1)
bn.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
3,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
4,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.60,12.43,22.9


Информация о датасете, просмотр пропущенных значений. У нас пропущеных значений нет

In [21]:
bn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 14 columns):
crim       333 non-null float64
zn         333 non-null float64
indus      333 non-null float64
chas       333 non-null int64
nox        333 non-null float64
rm         333 non-null float64
age        333 non-null float64
dis        333 non-null float64
rad        333 non-null int64
tax        333 non-null int64
ptratio    333 non-null float64
black      333 non-null float64
lstat      333 non-null float64
medv       333 non-null float64
dtypes: float64(11), int64(3)
memory usage: 36.5 KB


Так как у имеющихся значений высокая корреляция, да и самих колонок не так много, то можем оставить датафрейм без изменений

In [22]:
corrmat = bn.corr()
fcor=corrmat.abs().unstack().sort_values(ascending=False).reset_index()
fcor.columns = ['Feature_1', 'Feature_2','r']
fcor.query('Feature_2 !="diagnosis" & Feature_1 == "medv"').head(10)

,Feature_1,Feature_2,r
0,medv,medv,1.000000
22,medv,lstat,0.738600
30,medv,rm,0.689598
72,medv,ptratio,0.481376
78,medv,indus,0.473932
87,medv,tax,0.448078
95,medv,nox,0.413054
97,medv,crim,0.407454
118,medv,age,0.358888
125,medv,rad,0.352251


Нормализуем данные, а также исключим прогнозируемую колонку medv

In [29]:
X = bn.drop(['medv'], axis=1)
y = bn.medv

In [43]:
col = X.columns
X = StandardScaler().fit_transform(X)
X = pd.DataFrame(X, columns=col).astype('float64')
X.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
0,-0.456875,0.322906,-1.285631,-0.25278,-0.166787,0.440154,-0.107736,0.192133,-0.98907,-0.664062,-1.465171,0.432990,-1.067772
1,-0.454016,-0.472123,-0.604425,-0.25278,-0.767928,0.221060,0.379963,0.635528,-0.87451,-0.980620,-0.301616,0.432990,-0.478299
2,-0.453327,-0.472123,-1.304235,-0.25278,-0.863762,1.041952,-0.798347,1.189127,-0.75995,-1.097863,0.117264,0.406733,-1.356840
3,-0.448330,-0.472123,-1.304235,-0.25278,-0.863762,1.253933,-0.499320,1.189127,-0.75995,-1.097863,0.117264,0.432990,-1.018177
4,-0.445709,0.079980,-0.489936,-0.25278,-0.288758,-0.360821,-0.057898,0.935505,-0.53083,-0.576130,-1.511713,0.417953,-0.012106


Разделим на трейн и тест

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Решим задачу методом линейной регрессии

In [31]:
bnm = LinearRegression()

In [32]:
bnm.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [33]:
bnm.intercept_, bnm.coef_

(25.45398280437683,
 array([-2.35963335e-02,  5.22049640e-02,  4.07677738e-02,  2.70891600e+00,
        -1.22719027e+01,  4.67308377e+00, -7.63623392e-03, -1.50886949e+00,
         2.64523383e-01, -1.06490040e-02, -7.75373679e-01,  9.15075908e-03,
        -5.64205190e-01]))

In [34]:
y_pred=bnm.predict(X_test)
bnm.score(X_test,y_test)

0.6647488510185884

In [35]:
y_predicted_prob = bnm.predict(X_test)
y_predicted_prob

array([14.03523164, 11.3307701 , 36.60825889, 19.84781941, 19.47150274,
       19.92905659, 25.5989315 , 12.93921866, 25.35668038, 22.70899348,
       18.00299778, 16.83928647, 29.00939761, 19.10467293, 33.37522178,
       23.49006286, 21.19051049, 32.11997472, 39.98370383, 24.45683062,
       14.24946278, 18.00138404, 34.85665759, 17.01513186, 23.25755376,
       38.7059742 , 16.77246386,  8.59573053, 20.51390076, 34.04412472,
       11.79574807, 33.06250353, 16.90707074, 25.49287727, 22.98483043,
       14.58158266, 29.85331214, 19.40336544, 22.2666031 , 20.15973417,
       14.31066727, 28.06730324, 31.4246925 , 24.21856021, 25.11852398,
       27.32071452, 26.85333084, 32.41308516, 14.81789455, 24.95996604,
        9.88782574, 26.87774276, 20.42650117, 27.34082951, 21.95400662,
       33.39661566, 30.83954031, 28.56950961, 20.20076788, 28.52810682,
       29.06548588, 24.5201477 , 24.45787658, 21.78010747, 18.32431541,
       23.64354501,  7.05768207, 22.7088485 , 14.69645133, 23.97

In [36]:
mean_squared_error(y_test, y_pred)

25.625908551776796

In [37]:
r2_score(y_test, y_pred)

0.6647488510185884

In [38]:
mean_absolute_error(y_test, y_pred)

3.4021524108381693